In [231]:
%%time
# 筛选真正的记者职位
import pandas as pd
from string import punctuation
from dateutil import parser
from datetime import timedelta
import time
punctuation += '\"“”‘’—-–'

def kill_punctuations_capitals(string): 
    string = string.replace("’s","") 
    translator = str.maketrans("","",punctuation) 
    return string.lower().translate(translator)

def check_real(string, real_check_list):
    j = 0
    for i in real_check_list:
        j += kill_punctuations_capitals(string).find(i)
    return j > -len(real_check_list)

df=pd.read_csv("0 jobs.csv")

real_check_list = ['journalist','reporter','editor','news']
real_flag = []
for i in range(0,df.iloc[:,0].size):
    real_flag.append(check_real(df['Title'][i], real_check_list))
df['Real']=real_flag

df_selected_jobs = df[(df['Real']==True)].sort_values(by='Post_Date',ascending=False).loc[:,'Title':'Description']
df_selected_jobs.to_csv('1 new-selected-jobs.csv',na_rep='NaN',index=False)

CPU times: user 97.2 ms, sys: 10.5 ms, total: 108 ms
Wall time: 108 ms


In [279]:
%%time
# 计数所有frequency
from string import punctuation
import pandas as pd

def count_frequency(text):
    def kill_punctuations_capitals(text): 
        global punctuation
        punctuation += '\"“”‘’—-–·`'
        text = text.replace("’s","") #需要先去除‘s，否則去除標點會留下如chinas，trumps這樣的詞
        translator = str.maketrans("","",punctuation) 
        return text.lower().translate(translator).split() #將大寫字母轉化為小寫，去除標點，列出單詞

    def extract_meaningful(list): #去除無意義的單詞
        list_meaningful_words = []
        with open ('stopword.txt','r') as s:
            list_stop_words = s.read().split() #讀取stoplist
        for m in list:
            if m not in list_stop_words:
                list_meaningful_words.append(m)
        return list_meaningful_words

    def words_frequency(list): #統計一篇文章中的frequency
        dict_words_frequency={}
        for m in list:
            dict_words_frequency[m]=list.count(m)
        return dict_words_frequency
    return words_frequency(extract_meaningful(kill_punctuations_capitals(text)))

def rank_frequency(dict): #根據frequency排序，也可以最後用print(s.sort_values(ascending=False))，但是不方便寫cvs
    dict_frequency_rank={}
    rank = sorted(dict.items(), key=lambda item: item[1], reverse=True)  #將字典轉化爲二元數組，並根據字典中value排序
    for m in range(0,len(rank)):
        dict_frequency_rank.update({rank[m][0]:rank[m][1]}) #再將字典重新整合起來
    return dict_frequency_rank

df_each_frequency = pd.read_csv("1 new-selected-jobs.csv")
list_each_frequency = []

for i in df_each_frequency['Description']:
    list_each_frequency.append(rank_frequency(count_frequency(i)))

df_each_frequency['Description'] = list_each_frequency
df_each_frequency.to_csv('2 jobs-with-frequency.csv')

CPU times: user 1.02 s, sys: 36.9 ms, total: 1.05 s
Wall time: 1.08 s


In [280]:
%%time
import pandas as pd
df=pd.read_csv("1 new-selected-jobs.csv")
data_flag = []
for i in df['Description']:
    data_flag.append(i.lower().find('data') != -1)
df['data'] = data_flag
df_refer_data = df[(df['data']==True)].loc[:,'Title':'Description']
df_refer_data.to_csv('3 jobs-refer_data.csv')

CPU times: user 34.1 ms, sys: 5.12 ms, total: 39.2 ms
Wall time: 37.7 ms


In [288]:
%%time
import pandas as pd

def update_dict(dict0,dict1): #兩個txt中的frequency相加
    for k,v in dict1.items():
        if dict0.__contains__(k):
            dict0[k] += v
        else:
            dict0.update({k : dict1[k]})

def rank_frequency(dict): #根據frequency排序，也可以最後用print(s.sort_values(ascending=False))，但是不方便寫cvs
    dict_frequency_rank={}
    rank = sorted(dict.items(), key=lambda item: item[1], reverse=True)  #將字典轉化爲二元數組，並根據字典中value排序
    for m in range(0,len(rank)):
        dict_frequency_rank.update({rank[m][0]:rank[m][1]}) #再將字典重新整合起來
    return dict_frequency_rank

df_each_frequency = pd.read_csv("2 jobs-with-frequency.csv")
dict_accumulate_frequency = {} 
for i in df_each_frequency['Description']:
    update_dict(dict_accumulate_frequency,eval(i))
dict_frequency_rank = rank_frequency(dict_accumulate_frequency) #排序

df_frequency=pd.DataFrame(list(dict_frequency_rank.items()),columns=['word','frequency'])
df_frequency.index=range(1,len(df_frequency) + 1)
df_frequency.to_csv('4 description-frequency-rank.csv')

CPU times: user 248 ms, sys: 6.22 ms, total: 255 ms
Wall time: 256 ms
